In [58]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer, pipeline
from huggingface_hub import login, notebook_login
import bitsandbytes as bnb
import torch

import gradio as gr
import pypdf
from IPython.display import display, Markdown
from pathlib import Path

In [41]:
def print_markdown(text):
    """Display text as Markdown"""
    display(Markdown(text))

In [42]:
torch.set_default_device("cuda")
print(torch.cuda.get_device_name(0))

NVIDIA GeForce RTX 4070 SUPER


In [43]:
notebook_login()

In [44]:
# model_id_1 = "unsloth/gemma-3-4b-it-GGUF"
model_id_2 = "microsoft/Phi-4-mini-instruct"
# model_id_3 = "meta-llama/Llama-3.1-8B-Instruct"

In [45]:
tokenizer = AutoTokenizer.from_pretrained(model_id_2, trust_remote_code=True)

load_in_4bit=True: This enables 4-bit quantization for the model. When set to True, it instructs the loader to quantize the model weights to 4 bits instead of the usual 32-bit floating-point representation. This significantly reduces the memory footprint of the model, allowing larger models to fit into limited GPU memory. It’s important to note that this is a trade-off between model size/speed and potential slight loss in accuracy.

bnb_4bit_quant_type=”nf4": This specifies the type of 4-bit quantization to use. “nf4” stands for “Normal Float 4”, a quantization method designed to be more suitable for language models compared to standard 4-bit quantization. It uses a normal (Gaussian) distribution to represent weights, which often matches the distribution of weights in neural networks better than uniform quantization.

bnb_4bit_compute_dtype=”float16": This sets the data type used for computations during inference or training. “float16” specifies that calculations should be done in half-precision floating-point format. Using float16 can speed up computations and reduce memory usage compared to float32, but with a potential small loss in precision. It’s generally considered a good balance between performance and accuracy for many applications.

bnb_4bit_use_double_quant=True: Enables nested quantization, also known as “double quantization”. When set to True, it applies a second level of quantization to the already quantized weights. The first quantization reduces weights to 4 bits, and the second quantization further compresses the scales and zeros of the first quantization. This can provide additional memory savings with minimal impact on model quality.

In [46]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
    )

In [48]:
model = AutoModelForCausalLM.from_pretrained(
    model_id_2,
    quantization_config=quantization_config,
    # device_map="auto",
    trust_remote_code=True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [49]:
prompt = "Explain how large language model fine-tuning works in an easy to understand way."

In [50]:
# Return a tokenized versin of the input prompt for model input
inputs = tokenizer(prompt, return_tensors="pt")

In [51]:
outputs = model.generate(**inputs, max_new_tokens=1000)

In [53]:
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print_markdown(response)

Explain how large language model fine-tuning works in an easy to understand way. Large language models (LLMs) like me are trained on vast amounts of text data to understand and generate human-like text. Fine-tuning is a process where we take a pre-trained LLM and adapt it to a specific task or domain, improving its performance on that particular task.

Imagine you have a large language model that has been trained on general text data. This model can perform a wide range of tasks, like answering questions, writing essays, or translating languages. However, it might not be very good at performing tasks specific to a particular domain, like medical terminology or legal jargon.

Fine-tuning helps us adapt the LLM to a specific domain or task. Here's how it works in an easy-to-understand way:

1. Choose a pre-trained LLM: Start with a pre-trained LLM like me, which has been trained on a large corpus of text data.

2. Collect domain-specific data: Gather a dataset that is representative of the domain or task you want the LLM to excel at. For example, if you want the LLM to perform well in medical terminology, gather medical texts, research papers, and clinical notes.

3. Prepare the data: Format the collected data in a way that the LLM can understand. This usually involves splitting the data into training and validation sets, and possibly creating a vocabulary or tokenization scheme.

4. Fine-tune the LLM: Feed the prepared data into the LLM, allowing it to learn from the domain-specific examples. During this process, the LLM's parameters are updated to better fit the new data. This step is called "fine-tuning."

5. Evaluate and test: After fine-tuning, evaluate the LLM's performance on the validation set to ensure it has learned the domain-specific knowledge. Fine-tuning can also involve hyperparameter tuning and other optimization techniques to improve performance.

6. Use the fine-tuned LLM: Once fine-tuning is complete, the LLM is now better equipped to perform tasks specific to the domain or task it was fine-tuned on. You can now use the fine-tuned LLM for your specific use case, like generating medical reports, answering medical questions, or translating medical documents.

Fine-tuning allows us to leverage the powerful capabilities of large language models while adapting them to specific domains or tasks, improving their performance and making them more useful for our particular needs.

In [55]:
pipe = pipeline(
    "text-generation",
    model = model_id_2,
    tokenizer = tokenizer,
    device_map = "auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.
Device set to use cuda:0


In [56]:
outputs = pipe(
    prompt,
    max_new_tokens=1000,
    temperature=1,
)

In [57]:
print_markdown(outputs[0]['generated_text'])

Explain how large language model fine-tuning works in an easy to understand way. Large language models (LLMs) like GPT (Generative Pre-trained Transformer) are pre-trained on massive datasets and later "fine-tuned" on specific tasks or datasets to improve their performance in certain domains. Fine-tuning is the process of adjusting these models to better capture nuances and understand the context of a particular topic or domain.

Here's a step-by-step simplified explanation of large language model fine-tuning:

1. Start with a pre-trained model:
Imagine a large LLM as a powerful engine that you've pre-installed in your car. This engine has already learned basic skills like driving, stopping, and handling curves from countless examples of other cars' driving patterns. In the case of LLMs, they are like these engines, and they've learned to understand and generate human-like text based on the vast amount of text they're trained on.

2. Understand the goal:
Before you continue fine-tuning, it's essential to understand the specific task or domain you're focusing on. For instance, if you want your car (LLM) to become an expert in composing lyrics, you'll need to understand it better than before.

3. Collect a specialized dataset:
Fine-tuning demands a smaller dataset that's more relevant to the task. Imagine if you have access to a racing track where various kinds of cars performed specific maneuvers. This track data helps you understand how an expert car performs under different conditions. Similarly, you would gather a specialized dataset that represents the language, style, and context of the desired domain—a collection of texts aligned with the goal of the fine-tuned model, like lyrics or legal documents.

4. Adjust the pre-trained model:
The core LLM is like your pre-trained car engine, and fine-tuning adjusts it by incorporating information from the specialized dataset. It's akin to upgrading the racing cars' knowledge about new racing tracks and maneuvers. These adjustments can be thought of as the engine learning how to handle new situations, and it involves updating the weights in the neural network that formed the foundation of the pre-trained model. This process also involves adjusting the gradients during training to adapt to the new context provided by the fine-tuning dataset.

5. Train on the specific task:
Imagine now that you enter the racing track with your newly upgraded car. You begin practicing on this circuit, improving your skills, and adapting your driving style to perform better than you initially did. During fine-tuning, the LLM is trained similarly but now specifically on your racing track. Each iteration in the training rounds helps the model understand the nuances and intricacies of the domain better.

6. Validate the improvements:
After practicing on the racing track, you validate your improvements by driving in similar tracks elsewhere. Likewise, verifying the fine-tuning by running the model on a separate dataset is essential. This step often includes evaluating using metrics from relevant natural language processing (NLP) benchmarks, such as F1-score for Named Entity Recognition (NER) or BLEU score for machine translation. Such metric evaluations help gauge the performance of your upgraded model.

7. Use the fine-tuned model:
With your enhanced skills on the racing track, you are now ready to race confidently, outpacing other drivers using the general training you've received. Finally, the fine-tuned LLM is then used for the task in which it has been fine-tuned, like creating new lyrics, translating texts, or other language-based tasks.

In essence, fine-tuning is when a large language model with general language skills is specifically trained on a smaller dataset relevant to a task, helping it get even better at that task by incorporating the new information and context. Fine-tuning effectively transforms the general-purpose engine into an advanced performer specializing in the designated tracks or tasks.